In [27]:
import tensorflow as tf
import matplotlib.pyplot as plt

from glob import glob
import pandas as pd
import datetime
import numpy as np
import math

#from pyramid.arima import auto_arima
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error

import time

In [2]:
"""
read data (processed_data)
"""
idxname = pd.read_csv("data/index.csv", sep='\t')
stkname = pd.read_csv("data/stocks.csv", sep='\t')

indicators = []
stocks = []
for filename in idxname.filename:
    indicators.append(pd.read_csv("processed_data/index/"+filename))
for filename in stkname.symbol:
    stocks.append(pd.read_csv("processed_data/stocks/"+filename))

#print(indicators[0])
#print(stocks[0])

"""
Data preparation
"""
max0day = int(stocks[0].loc[stocks[0].iloc[:,1]==0].Date.max())

#stocks[0].iloc[max0day:,1:5].plot()
#plt.show()
data = stocks[0].iloc[max0day+1:].Close

train = data.iloc[:-730]
test = data.iloc[-730:]

In [28]:
"""
read data (original)
"""
idxname = pd.read_csv("data/index.csv", sep='\t')
stkname = pd.read_csv("data/stocks.csv", sep='\t')

indicators = []
stocks = []
for filename in idxname.filename:
    indicators.append(pd.read_csv("data/index/"+filename).fillna(0))
for filename in stkname.symbol:
    stocks.append(pd.read_csv("data/stocks/"+filename).fillna(0))

#print(indicators[0])
#print(stocks[0])

"""
Data preparation
"""
data = stocks[0].Close

train = data.iloc[:-730]
test = data.iloc[-730:]

In [29]:
"""
ARIMA
https://github.com/pierpaolo28/Kaggle-Challenges/blob/master/stock-market-analysis-and-time-series-prediction.ipynb
"""
start = time.time()
#stepwise_model = auto_arima(train)
# stepwise_model = auto_arima(train, start_p=1, start_q=1,
#                            max_p=3, max_q=3, m=12,
#                            start_P=0, seasonal=False,
#                            d=1, trace=True,
#                            error_action='ignore',  
#                            suppress_warnings=True, 
#                            stepwise=True)

# print(stepwise_model.aic())

train_ar = train.values.tolist()
test_ar = test.values.tolist()

history = train_ar.copy()
predictions = []

print(len(test_ar))
for i in range(len(test_ar)):
    model = ARIMA(history, order=(5,1,0))
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test_ar[i]
    history.append(obs)
print('Execution time:', time.time()-start)

730
Execution time: 133.91602730751038


In [30]:
err = mean_squared_error(test_ar, predictions)
print('Testing Mean Squared Error: %.3f' % err)

Testing Mean Squared Error: 7.177


In [31]:
def smape_kun(y_true, y_pred):
    return np.mean((np.abs(np.array(y_pred) - np.array(y_true)) * 200/ (np.abs(y_pred) + np.abs(y_true))))
error2 = smape_kun(test_ar, predictions)
print('Symmetric mean absolute percentage error: %.3f' % error2)

Symmetric mean absolute percentage error: 21.381


In [32]:
#plt.figure(figsize=(12,7))
%matplotlib qt

#plt.plot(data, 'green', color='blue', label='Training Data')
plt.plot(test.index, predictions, 
         color='green', 
         marker=',', #'o', 
         linestyle='dashed', 
         label='Predicted Price')
# plt.plot(predictions, 
#          color='green', 
#          marker=',', #'o', 
#          linestyle='dashed', 
#          label='Predicted Price')
plt.plot(test.index, test, color='red', label='Actual Price')
# plt.plot(test, color='red', label='Actual Price')
plt.title('Prices Prediction')
plt.xlabel('Dates')
plt.ylabel('Prices')
#plt.xticks(np.arange(0,1857, 300), stocks[0]['Date'][0:1857:300])
#plt.xticks(np.arange(max0day+1,len(data), 300), stocks[0]['Date'][max0day+1:len(data):300])
plt.legend()

IndexError: tuple index out of range

The graph shows that the trained model just picks the same price of today as the price of the next day.